In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [2]:
# 全量数据

train_data = pd.read_csv('./data_format1/train_format1.csv')
test_data = pd.read_csv('./data_format1/test_format1.csv')
user_info = pd.read_csv('./data_format1/user_info_format1.csv')
user_log = pd.read_csv('./data_format1/user_log_format1.csv')


In [3]:
# 样本提取
'''
user_info.set_index('user_id', inplace=True)
user_log.set_index('user_id', inplace=True)
train_data_sample = train_data.sample(n=15000)
test_data_sample = test_data.sample(n=5000)
sample_user_id = train_data_sample['user_id'].tolist() + test_data_sample['user_id'].tolist()
user_info_sample = user_info.loc[sample_user_id]
user_log_sample = user_log.loc[sample_user_id]
train_data_sample.set_index('user_id', inplace=True)
test_data_sample.set_index('user_id', inplace=True)
'''

In [4]:
# 保存样本
'''
train_data_sample.to_csv('./sample/train_data_sample.csv')
test_data_sample.to_csv('./sample/test_data_sample.csv')
user_info_sample.to_csv('./sample/user_info_sample.csv')
user_log_sample.to_csv('./sample/user_log_sample.csv')
'''

In [2]:
# 加载样本
'''
train_data = pd.read_csv('./sample/train_data_sample.csv')
test_data = pd.read_csv('./sample/test_data_sample.csv')
user_info = pd.read_csv('./sample/user_info_sample.csv')
user_log = pd.read_csv('./sample/user_log_sample.csv')
'''

In [3]:
user_log.rename(columns={'seller_id': 'merchant_id'}, inplace=True)

In [4]:
# 数据清洗
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype('int64')
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%H%M')
user_info['age_range'].fillna(0, inplace=True)
user_info = user_info.replace([8], [7])
user_info['age_range'] = user_info['age_range'].astype('int64')
user_info['gender'].fillna(2, inplace=True)
user_info['gender'] = user_info['gender'].astype('int64')

In [5]:
print(train_data.info())
print(user_info.info())
print(user_log.info(null_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260864 entries, 0 to 260863
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   user_id      260864 non-null  int64
 1   merchant_id  260864 non-null  int64
 2   label        260864 non-null  int64
dtypes: int64(3)
memory usage: 6.0 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    424170 non-null  int64
 1   age_range  424170 non-null  int64
 2   gender     424170 non-null  int64
dtypes: int64(3)
memory usage: 9.7 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54925330 entries, 0 to 54925329
Data columns (total 7 columns):
 #   Column       Non-Null Count     Dtype         
---  ------       --------------     -----         
 0   user_id      54925330 non-null  int64         
 1   item_id      549253

In [6]:
train_data['origin'] = 'train'
test_data['origin'] = 'test'
matrix = pd.concat([train_data, test_data], ignore_index=True, sort=False)
matrix.drop('prob', axis=1, inplace=True)
matrix = matrix.merge(user_info, on='user_id', how='left')

In [7]:
# 用户特征处理
groups = user_log.groupby(['user_id'])
temp = groups.size().reset_index().rename(columns={0: 'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'u7', 1: 'u8', 2: 'u9', 3: 'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')

# 商户特征处理
groups = user_log.groupby(['merchant_id'])
temp = groups.size().reset_index().rename(columns={0: 'm1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id': 'm2', 'item_id': 'm3', 'cat_id': 'm4', 'brand_id': 'm5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'm6', 1: 'm7', 2: 'm8', 3: 'm9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = train_data[train_data['label'] == -1].groupby('merchant_id').size().reset_index().rename(columns={0: 'm10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

/Users/paulhwang/anaconda3/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [8]:
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0: 'um1'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id': 'um2', 'cat_id': 'um3', 'brand_id': 'um4'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'um5', 1: 'um6', 2: 'um7', 3: 'um8'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.seconds/3600
temp.drop(['last', 'first'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')

/Users/paulhwang/anaconda3/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


In [9]:
matrix['r1'] = matrix['u9'] / matrix['u7']
matrix['r2'] = matrix['m8'] / matrix['m6']
matrix['r3'] = matrix['um7'] / matrix['um5']
matrix.fillna(0, inplace=True)
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='gen')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)

In [10]:
train_data = matrix[matrix['origin'] == 'train'].drop('origin', axis=1)
test = matrix[matrix['origin'] == 'test'].drop(['origin', 'label'], axis=1)
x_train, y_train = train_data.drop('label', axis=1), train_data['label']

In [11]:
train_x, vali_x, train_y, vali_y = train_test_split(x_train, y_train, test_size=0.2)
model = xgb.XGBClassifier(max_depth=8, n_estimators=1000, min_child_wight=300, colsample_bytree=0.8, subsample=0.8, eta=0.3, seed=42)
model.fit(train_x, train_y, eval_metric='auc', eval_set=[(train_x, train_y), (vali_x, vali_y)], verbose=True, early_stopping_rounds=10)

[0]	validation_0-auc:0.640864	validation_1-auc:0.639882
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.649406	validation_1-auc:0.644581
[2]	validation_0-auc:0.656326	validation_1-auc:0.649942
[3]	validation_0-auc:0.662931	validation_1-auc:0.654453
[4]	validation_0-auc:0.666696	validation_1-auc:0.655364
[5]	validation_0-auc:0.672409	validation_1-auc:0.656128
[6]	validation_0-auc:0.674831	validation_1-auc:0.656422
[7]	validation_0-auc:0.676874	validation_1-auc:0.657238
[8]	validation_0-auc:0.67853	validation_1-auc:0.657452
[9]	validation_0-auc:0.680377	validation_1-auc:0.656899
[10]	validation_0-auc:0.684384	validation_1-auc:0.657365
[11]	validation_0-auc:0.686982	validation_1-auc:0.658591
[12]	validation_0-auc:0.688232	validation_1-auc:0.65824
[13]	validation_0-auc:0.691135	validation_1-auc:0.659435
[14]	validation_0-auc:0.692572	validation_1-auc:0.65925
[

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=1, min_child_wight=300, missing=None,
              n_estimators=1000, n_jobs=1, nthread=None,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=42, silent=None,
              subsample=0.8, verbosity=1)

In [12]:
prob = model.predict_proba(test)
test_data['prob'] = pd.Series(prob[:, 1])
test_data.drop(['origin'], axis=1, inplace=True)

In [13]:
test_data.to_csv('prediction.csv', index=False)